In [156]:
import csv
import pandas as pd

In [157]:
from mysql.connector import MySQLConnection, Error

In [158]:
#Making connection to SQL server
def make_connection():
    try:
        conn = MySQLConnection(host='IES-ADS-ClassDB.sjsu.edu',
                               database='querycrew_db',
                               user='querycrew_user',
                               password='Pomegranate_746')
        
        if conn.is_connected():
            print('Connected to the MySQL database!')
            
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [159]:
conn = make_connection()
cursor = conn.cursor()

Connected to the MySQL database!


In [160]:
cursor.execute('DROP TABLE IF EXISTS location')

In [161]:
cursor.execute('DROP TABLE IF EXISTS vehicle')

In [162]:
#This is for the one-to-one tables. The primary key is VIN(1-10) 
#Creating a vehicle table

sql = ( """
        CREATE TABLE vehicle
        (
            vin varchar(32),
            make varchar(32),
            model varchar(32),
            model_year int,
            PRIMARY KEY(vin)
        )
        """
      )
cursor.execute(sql)

In [163]:
#FIGURE OUT A PRIMARY KEY FOR THIS TABLE
sql = ( """
        CREATE TABLE location
        (
            dol_id int,
            vin varchar(20),
            county varchar(32),
            city varchar(32),
            state varchar(32),
            zip_code int,
            PRIMARY KEY(dol_id),
            FOREIGN KEY (vin) REFERENCES vehicle(vin)
        )
        """
      )

cursor.execute(sql);

In [164]:
#For both of these two tables created, the primary key is the vin, and the foriegn key is in the location table and it is the vin as well

In [165]:
sql_insert_vehicle = (  
    """
    INSERT INTO vehicle (vin, model_year, make, model)
    VALUES (%s, %s, %s, %s)
    """
)

In [166]:
sql_insert_location = ( 
    """
    INSERT INTO location (dol_id, vin, county, city, state, zip_code)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
)

In [167]:
def transform_vehicle(row):
    #Converting the model_year column and zip-code column to an int
    try:
      row[1] = int(row[1])
    except ValueError:
      row[4] = None
      row[1] = int(row[1])

    return row

In [168]:
def transform_location(row):
    try:
      row[0] = int(row[0])
      row[4] = int(row[4])
    except ValueError:
      row[0] = None
      row[4] = None

    return row

In [169]:
vehicle_cols = [0, 5, 6, 7]
location_cols = [13, 0, 1, 2, 3, 4]
first = True
seen_vins = set()  

# Read CSV data into a list before processing
with open('Electric_Vehicle_Population_Data.csv', newline='') as csv_file:
    data = list(csv.reader(csv_file, delimiter=',', quotechar='"'))  

for row in data:
    if not first:
        vin = row[vehicle_cols[0]]  

        # Skips over the duplicates found in the file
        if vin in seen_vins:
            continue  
        seen_vins.add(vin)  

        # Here I am checking if the VIN already exists within the database for vehicle
        cursor.execute("SELECT COUNT(*) FROM vehicle WHERE vin = %s", (vin,))
        if cursor.fetchone()[0] > 0:
            continue 
        #This is where we are identifying the specific columns 
        vehicle_row = [row[i] for i in vehicle_cols]
        location_row = [row[i] for i in location_cols]
          
        #Inserting into repsective tables
        cursor.execute(sql_insert_vehicle, vehicle_row)
        cursor.execute(sql_insert_location, location_row)

    first = False  

conn.commit()


In [170]:
sql = (  
    """
    SELECT * FROM vehicle LIMIT 25
    """
)
cursor.execute(sql)
results = cursor.fetchall()



for row in results:
    print(row)

conn.commit()

('1C4JJXN60P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXN61P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXN62P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXN63P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXN64P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXN65P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXN66P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXN67P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXN68P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXN69P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXN6XP', 'JEEP', 'WRANGLER', 2023)
('1C4JJXP60M', 'JEEP', 'WRANGLER', 2021)
('1C4JJXP60N', 'JEEP', 'WRANGLER', 2022)
('1C4JJXP60P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXP61M', 'JEEP', 'WRANGLER', 2021)
('1C4JJXP61N', 'JEEP', 'WRANGLER', 2022)
('1C4JJXP61P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXP62M', 'JEEP', 'WRANGLER', 2021)
('1C4JJXP62N', 'JEEP', 'WRANGLER', 2022)
('1C4JJXP62P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXP63M', 'JEEP', 'WRANGLER', 2021)
('1C4JJXP63N', 'JEEP', 'WRANGLER', 2022)
('1C4JJXP63P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXP64M', 'JEEP', 'WRANGLER', 2021)
('1C4JJXP64N', '

In [171]:
sql = (  
    """
    SELECT * FROM location LIMIT 25
    """
)
cursor.execute(sql)
results = cursor.fetchall()

if results:
    for row in results:
        print(row)
conn.commit()

(4469, '1G1RB6S51J', 'Jefferson', 'Port Townsend', 'WA', 98368)
(201400, 'KNDJP3AE5G', 'King', 'Federal Way', 'WA', 98023)
(206762, 'WBY1Z2C51E', 'King', 'Seattle', 'WA', 98118)
(296957, '5YJSA1E41G', 'Kitsap', 'Bainbridge Island', 'WA', 98110)
(429173, '1G1RB6S55J', 'King', 'Seattle', 'WA', 98177)
(546941, '1G1RB6S50H', 'King', 'Seattle', 'WA', 98122)
(1238030, 'JA3215H48G', 'Spokane', 'Spokane Valley', 'WA', 99216)
(1415298, '1G1RH6E4XE', 'King', 'Seattle', 'WA', 98103)
(1495133, 'KMHC05LH0H', 'King', 'Kirkland', 'WA', 98033)
(1562389, '1G1RF6E46D', 'Clark', 'Vancouver', 'WA', 98685)
(1593721, '3C3CFFGE4E', 'Yakima', 'Yakima', 'WA', 98902)
(1708409, '1N4BZ0CP4H', 'King', 'Shoreline', 'WA', 98155)
(1802146, '1G6RM1E42G', 'Snohomish', 'Stanwood', 'WA', 98292)
(1829285, 'WBY2Z6C54K', 'King', 'Sammamish', 'WA', 98074)
(1843054, '5YJXCAE29L', 'Kitsap', 'Silverdale', 'WA', 98383)
(1869563, 'KMHE14L29H', 'King', 'Seattle', 'WA', 98146)
(1871509, '1G1RA6S55H', 'King', 'Seattle', 'WA', 98133)

In [172]:
cursor.close()
conn.close()

In [173]:
#Look at the last values and make sure that they are separate cars